In [1]:
import joblib
import time
import mysql.connector
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as plt
sns.set(rc={'figure.figsize': [9, 9]}, font_scale=1.2)


In [2]:

df = pd.read_csv('lastKnew.csv')
df


C:\Users\Commandor\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,url,title,date,location,compound,type,price,area,bedrooms,level,...,landline,pets_allowed,covered_parking,private_garden,pool,maids_room,central_ac_heating,built_in_kitchen_appliances,month,meterprice
0,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بسكن مصر الاندلس مسلسل 1,2022-02-28,New Cairo - El Tagamoa,Sakan Masr,Apartment,175000.0,115,3,3,...,1,1,1,1,0,0,0,0,2,1521
1,https://www.olx.com.eg/en/ad/%D8%B3%D9%83%D9%8...,سكن مصر الأندلس مرحله ١ للبيع شقه دور اول ١١٥,2022-04-15,New Cairo - El Tagamoa,Sakan Masr,Apartment,180000.0,115,3,1,...,0,0,0,0,0,0,0,0,4,1565
2,https://www.olx.com.eg/en/ad/%D9%84%D9%82%D8%B...,لقطه سكن مصر التجمع التالت افر 150 الف,2022-04-29,New Cairo - El Tagamoa,Sakan Masr,Apartment,150000.0,115,3,2,...,0,0,0,0,0,0,0,0,4,1304
3,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بمدينة عمر بن الخطاب - آخر جسر السويس,2022-04-27,Gesr Al Suez,Not in Compound,Apartment,450000.0,135,3,7,...,0,0,0,0,0,0,0,0,4,3333
4,https://www.olx.com.eg/en/ad/%D9%84%D8%B3%D8%B...,لسرعه البيع شقه سكن مصر التجمع التالت,2022-04-29,New Cairo - El Tagamoa,Sakan Masr,Apartment,160000.0,115,3,2,...,0,0,0,0,0,0,0,0,4,1391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37704,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع إمتداد مدينة 15 مايو مساحة 125م,2022-04-29,15 May City,Not in Compound,Apartment,550000.0,125,3,2,...,0,0,0,0,0,0,0,0,4,4400
37705,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بكمبوند دار مصر دور ثالث,2022-04-29,15 May City,Dar Misr,Apartment,600000.0,140,3,3,...,0,0,0,0,0,0,0,0,4,4285
37706,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D9%8...,شقه للبيع,2022-02-28,15 May City,Jewar,Apartment,470000.0,80,2,5,...,0,0,0,0,0,0,0,0,2,5875
37707,https://www.olx.com.eg/en/ad/%D9%84%D9%84%D8%A...,للبيع,2022-02-28,15 May City,Not in Compound,Apartment,470000.0,100,2,4,...,1,1,0,1,0,0,0,0,2,4700


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37709 entries, 0 to 37708
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   url                          37709 non-null  object 
 1   title                        37709 non-null  object 
 2   date                         37709 non-null  object 
 3   location                     37709 non-null  object 
 4   compound                     37709 non-null  object 
 5   type                         37709 non-null  object 
 6   price                        37709 non-null  float64
 7   area                         37709 non-null  int64  
 8   bedrooms                     37709 non-null  object 
 9   level                        37709 non-null  object 
 10  furnished                    37709 non-null  int64  
 11  price_type                   37709 non-null  object 
 12  bathrooms                    37709 non-null  object 
 13  payment_option  

In [45]:
df['meterprice']=df['price']/df['area']


In [46]:
df[df['meterprice']]

count     37709.000000
mean       9216.959440
std        9091.484546
min           0.385000
25%        4222.222222
50%        7700.000000
75%       12222.222222
max      970588.235294
Name: meterprice, dtype: float64

In [3]:
def nega(row):
    if row['down_payment'] < 0 :
        return 0
    else:
        return row['down_payment']

df['down_payment'] = df.apply(nega, axis=1) 

In [4]:
df

,url,title,date,location,compound,type,price,area,bedrooms,level,...,landline,pets_allowed,covered_parking,private_garden,pool,maids_room,central_ac_heating,built_in_kitchen_appliances,month,meterprice
0,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بسكن مصر الاندلس مسلسل 1,2022-02-28,New Cairo - El Tagamoa,Sakan Masr,Apartment,175000,115,3,3,...,1,1,1,1,0,0,0,0,2,1521
1,https://www.olx.com.eg/en/ad/%D8%B3%D9%83%D9%8...,سكن مصر الأندلس مرحله ١ للبيع شقه دور اول ١١٥,2022-04-14,New Cairo - El Tagamoa,Sakan Masr,Apartment,180000,115,3,1,...,0,0,0,0,0,0,0,0,4,1565
2,https://www.olx.com.eg/en/ad/%D9%84%D9%82%D8%B...,لقطه سكن مصر التجمع التالت افر 150 الف,2022-04-28,New Cairo - El Tagamoa,Sakan Masr,Apartment,150000,115,3,2,...,0,0,0,0,0,0,0,0,4,1304
3,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بمدينة عمر بن الخطاب - آخر جسر السويس,2022-04-26,Gesr Al Suez,no_compound,Apartment,450000,135,3,7,...,0,0,0,0,0,0,0,0,4,3333
4,https://www.olx.com.eg/en/ad/%D9%84%D8%B3%D8%B...,لسرعه البيع شقه سكن مصر التجمع التالت,2022-04-28,New Cairo - El Tagamoa,Sakan Masr,Apartment,160000,115,3,2,...,0,0,0,0,0,0,0,0,4,1391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37704,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع إمتداد مدينة 15 مايو مساحة 125م,2022-04-28,15 May City,no_compound,Apartment,550000,125,3,2,...,0,0,0,0,0,0,0,0,4,4400
37705,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بكمبوند دار مصر دور ثالث,2022-04-28,15 May City,Dar Misr,Apartment,600000,140,3,3,...,0,0,0,0,0,0,0,0,4,4285
37706,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D9%8...,شقه للبيع,2022-02-28,15 May City,Jewar,Apartment,470000,80,2,5,...,0,0,0,0,0,0,0,0,2,5875
37707,https://www.olx.com.eg/en/ad/%D9%84%D9%84%D8%A...,للبيع,2022-02-28,15 May City,no_compound,Apartment,470000,100,2,4,...,1,1,0,1,0,0,0,0,2,4700


In [13]:
ix=df.columns
list(ix)
ix

Index(['url', 'title', 'date', 'location', 'compound', 'type', 'price', 'area',
       'bedrooms', 'level', 'furnished', 'price_type', 'bathrooms',
       'payment_option', 'description', 'amenities', 'delivery_term',
       'delivery_date', 'down_payment', 'electricity_meter', 'balcony',
       'water_meter', 'elevator', 'security', 'natural_gas', 'landline',
       'pets_allowed', 'covered_parking', 'private_garden', 'pool',
       'maids_room', 'central_ac_heating', 'built_in_kitchen_appliances',
       'month', 'meterprice'],
      dtype='object')

In [19]:
list_um=ix[19:]
list_um

Index(['electricity_meter', 'balcony', 'water_meter', 'elevator', 'security',
       'natural_gas', 'landline', 'pets_allowed', 'covered_parking',
       'private_garden', 'pool', 'maids_room', 'central_ac_heating',
       'built_in_kitchen_appliances', 'month', 'meterprice'],
      dtype='object')

In [32]:
df.columns

Index(['url', 'title', 'date', 'location', 'compound', 'type', 'price', 'area',
       'bedrooms', 'level', 'furnished', 'price_type', 'bathrooms',
       'payment_option', 'description', 'amenities', 'delivery_term',
       'delivery_date', 'down_payment', 'electricity_meter', 'balcony',
       'water_meter', 'elevator', 'security', 'natural_gas', 'landline',
       'pets_allowed', 'covered_parking', 'private_garden', 'pool',
       'maids_room', 'central_ac_heating', 'built_in_kitchen_appliances',
       'month', 'meterprice'],
      dtype='object')

In [42]:
x=df['location'].unique()
list(x)
for x in df['location'].unique():
    print(f'<option value="{x}">{x}</option>')
    

<option value="New Cairo - El Tagamoa">New Cairo - El Tagamoa</option>
<option value="Gesr Al Suez">Gesr Al Suez</option>
<option value="Ain Shams">Ain Shams</option>
<option value="Nasr City">Nasr City</option>
<option value="Sayeda Zeinab">Sayeda Zeinab</option>
<option value="Katameya">Katameya</option>
<option value="Shorouk City">Shorouk City</option>
<option value="Hadayek Helwan">Hadayek Helwan</option>
<option value="Badr City">Badr City</option>
<option value="Salam City">Salam City</option>
<option value="Shubra">Shubra</option>
<option value="Rehab City">Rehab City</option>
<option value="New Capital City">New Capital City</option>
<option value="Mokattam">Mokattam</option>
<option value="Mostakbal City">Mostakbal City</option>
<option value="Hadayek al-Kobba">Hadayek al-Kobba</option>
<option value="Obour City">Obour City</option>
<option value="Madinaty">Madinaty</option>
<option value="Downtown Cairo">Downtown Cairo</option>
<option value="Maadi">Maadi</option>
<option va

In [39]:
for i in df['bedrooms'].unique():
    print(f'<option value="{i}">{i}</option>')

<option value="0">0</option>
<option value="1">1</option>
<option value="2">2</option>
<option value="3">3</option>
<option value="4">4</option>
<option value="5">5</option>
<option value="6">6</option>
<option value="7">7</option>
<option value="8">8</option>
<option value="9">9</option>
<option value="10">10</option>


In [ ]:
print(f'<option value="{saturday}">{Saturday}</option>')

In [33]:
df['area'].max()

268

In [30]:
list_um
for i in list_um:
    i.replace('_',' ')
    i.capitalize()
list_um    

Index(['electricity_meter', 'balcony', 'water_meter', 'elevator', 'security',
       'natural_gas', 'landline', 'pets_allowed', 'covered_parking',
       'private_garden', 'pool', 'maids_room', 'central_ac_heating',
       'built_in_kitchen_appliances', 'month', 'meterprice'],
      dtype='object')

In [5]:
df['furnished'].replace('No',0,inplace = True)
df['furnished'].replace('Yes',1,inplace = True)



In [4]:
df['furnished'].unique()


array([0, 1], dtype=int64)

In [7]:
df['delivery_date'].unique()

array(['Ready to move', 'soon', 'within 6 months', '2022', '2023', '2024',
       '2026', '2025', '2030', '2027', '2028'], dtype=object)

In [8]:
df.columns

Index(['url', 'title', 'date', 'location', 'compound', 'type', 'price', 'area',
       'bedrooms', 'level', 'furnished', 'price_type', 'bathrooms',
       'payment_option', 'description', 'amenities', 'delivery_term',
       'delivery_date', 'down_payment', 'electricity_meter', 'balcony',
       'water_meter', 'elevator', 'security', 'natural_gas', 'landline',
       'pets_allowed', 'covered_parking', 'private_garden', 'pool',
       'maids_room', 'central_ac_heating', 'built_in_kitchen_appliances',
       'month', 'meterprice'],
      dtype='object')

In [40]:
df['meterprice']=df['price']/df['area']

In [41]:
df['meterprice'].describe()

count     37709.000000
mean       9216.959440
std        9091.484546
min           0.385000
25%        4222.222222
50%        7700.000000
75%       12222.222222
max      970588.235294
Name: meterprice, dtype: float64

In [9]:
df = pd.get_dummies(df, columns=['delivery_term','location', 'compound', 'bedrooms', 'level','price_type','bathrooms','payment_option','delivery_date',], drop_first=True)
df

,url,title,date,type,price,area,furnished,description,amenities,down_payment,...,delivery_date_2023,delivery_date_2024,delivery_date_2025,delivery_date_2026,delivery_date_2027,delivery_date_2028,delivery_date_2030,delivery_date_Ready to move,delivery_date_soon,delivery_date_within 6 months
0,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بسكن مصر الاندلس مسلسل 1,2022-02-28,Apartment,175000,115,0,شقة لقطة للبيع بسكن مصر الاندلس دور متكرر فيو ...,Landline; Natural Gas; Water Meter; Electricit...,0,...,0,0,0,0,0,0,0,1,0,0
1,https://www.olx.com.eg/en/ad/%D8%B3%D9%83%D9%8...,سكن مصر الأندلس مرحله ١ للبيع شقه دور اول ١١٥,2022-04-14,Apartment,180000,115,0,للبيع دور اول\r\nفيو تحفه\r\nالأوفر ١٨٠ الف لس...,Natural Gas; Water Meter; Electricity Meter; S...,0,...,0,0,0,0,0,0,0,1,0,0
2,https://www.olx.com.eg/en/ad/%D9%84%D9%82%D8%B...,لقطه سكن مصر التجمع التالت افر 150 الف,2022-04-28,Apartment,150000,115,0,للبيت شقه سكن مصر التجمع التالت مسلسل 9\r\nمرح...,NaN,0,...,0,0,0,0,0,0,0,1,0,0
3,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بمدينة عمر بن الخطاب - آخر جسر السويس,2022-04-26,Apartment,450000,135,0,شقة عبارة عن نصف دور بها عدد 3 جهاز تكييف وسخا...,Elevator; Natural Gas; Balcony,0,...,0,0,0,0,0,0,0,1,0,0
4,https://www.olx.com.eg/en/ad/%D9%84%D8%B3%D8%B...,لسرعه البيع شقه سكن مصر التجمع التالت,2022-04-28,Apartment,160000,115,0,لقطه لسرعه البيع شقه سكن مصر التجمع التالت مسل...,NaN,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37704,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع إمتداد مدينة 15 مايو مساحة 125م,2022-04-28,Apartment,550000,125,0,شقة للبيع إمتداد مدينة 15 مايو مساحة 125م \...,NaN,0,...,0,0,0,0,0,0,0,1,0,0
37705,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D8%A...,شقة للبيع بكمبوند دار مصر دور ثالث,2022-04-28,Apartment,600000,140,0,شقة للبيع بكمبوند دار مصر 15 مايو دور ثالث بحر...,NaN,600000,...,0,0,0,0,0,0,0,1,0,0
37706,https://www.olx.com.eg/en/ad/%D8%B4%D9%82%D9%8...,شقه للبيع,2022-02-28,Apartment,470000,80,0,شقه للبيع بارقي مكان بالإسكندرية أبراج رمادا ب...,Elevator; Natural Gas; Water Meter; Electricit...,0,...,0,0,0,0,0,0,0,1,0,0
37707,https://www.olx.com.eg/en/ad/%D9%84%D9%84%D8%A...,للبيع,2022-02-28,Apartment,470000,100,0,شقة للبيع بمدينة ١٥ مايو مجاورة ٥ \r\nقريبة من...,Landline; Water Meter; Electricity Meter; Pets...,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
df.drop(['url','title','description','type','amenities','date'],axis=1,inplace=True)

In [4]:
df.columns

Index(['location', 'compound', 'price', 'area', 'bedrooms', 'level',
       'furnished', 'price_type', 'bathrooms', 'payment_option',
       'delivery_term', 'delivery_date', 'down_payment', 'electricity_meter',
       'balcony', 'water_meter', 'elevator', 'security', 'natural_gas',
       'landline', 'pets_allowed', 'covered_parking', 'private_garden', 'pool',
       'maids_room', 'central_ac_heating', 'built_in_kitchen_appliances',
       'month', 'meterprice'],
      dtype='object')

No
Not in Compound
Yes


In [12]:
df[df['compound']=='no_compound']='Not in Compound'


In [13]:
df[df['compound']=='no_compound']

,location,compound,price,area,bedrooms,level,furnished,price_type,bathrooms,payment_option,...,landline,pets_allowed,covered_parking,private_garden,pool,maids_room,central_ac_heating,built_in_kitchen_appliances,month,meterprice


In [11]:
df.columns

Index(['price', 'area', 'furnished', 'down_payment', 'electricity_meter',
       'balcony', 'water_meter', 'elevator', 'security', 'natural_gas',
       ...
       'delivery_date_2023', 'delivery_date_2024', 'delivery_date_2025',
       'delivery_date_2026', 'delivery_date_2027', 'delivery_date_2028',
       'delivery_date_2030', 'delivery_date_Ready to move',
       'delivery_date_soon', 'delivery_date_within 6 months'],
      dtype='object', length=406)

In [12]:
x = df.drop('meterprice', axis=1)
y = df['meterprice']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()

In [17]:
scaler.fit(x_train)

StandardScaler()

In [18]:
x_test

,price,area,furnished,down_payment,electricity_meter,balcony,water_meter,elevator,security,natural_gas,...,delivery_date_2023,delivery_date_2024,delivery_date_2025,delivery_date_2026,delivery_date_2027,delivery_date_2028,delivery_date_2030,delivery_date_Ready to move,delivery_date_soon,delivery_date_within 6 months
757,1600000,140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
26820,250000,80,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,1,0,0
25579,220000,80,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
8665,510000,151,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
33645,320000,110,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3712,1148000,162,0,150000,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
18660,3500000,200,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
25773,285000,75,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
31202,2340000,140,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,0


In [19]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [22]:
models = {
    "LR": LinearRegression(),
    "KNNR" : KNeighborsRegressor(), 
    "SVR": SVR(),
    "DT": DecisionTreeRegressor(),
    "RF": RandomForestRegressor(),
    "XGBR": XGBRegressor()
}

In [23]:
model = XGBRegressor()
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [29]:
for name, model in models.items():
    print(f'Using model: {name}')
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f'MASE Score: {np.sqrt(mean_absolute_error(y_test, y_pred))}')  
    print(f'MSE Score: {np.sqrt(mean_squared_error(y_test, y_pred))}')  
    print(f'R2 Score: {r2_score(y_test, y_pred)}')
    print('-'*30)

Using model: LR
MASE Score: 783349.0757441476
MSE Score: 24086486129437.95
R2 Score: -1.035760680049599e+19
------------------------------
Using model: KNNR
MASE Score: 49.271414631666005
MSE Score: 5118.60157264396
R2 Score: 0.5322485127615406
------------------------------
Using model: SVR
MASE Score: 68.4486805166331
MSE Score: 7510.344730233382
R2 Score: -0.007006489416597761
------------------------------
Using model: DT
MASE Score: 15.655688529990124
MSE Score: 11119.694473024583
R2 Score: -1.2074874071787862
------------------------------
Using model: RF
MASE Score: 10.21563200942327
MSE Score: 3414.1402076176128
R2 Score: 0.7918984907656333
------------------------------
Using model: XGBR
MASE Score: 11.864497155835744
MSE Score: 720.7145477107164
R2 Score: 0.990726597667021
------------------------------


In [30]:
df['meterprice'].describe()

count     37709.000000
mean       9216.584211
std        9091.470402
min           0.000000
25%        4222.000000
50%        7700.000000
75%       12222.000000
max      970588.000000
Name: meterprice, dtype: float64

In [ ]:
# y_pred = model.predict(x_test)
# np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
# r2_score(y_test, y_pred)

In [ ]:
# from sklearn.feature_selection import RFE, SelectFromModel

In [ ]:
# selector = RFE(model)
# selector.fit(x_train, y_train)

In [25]:
KNNR=KNeighborsRegressor()


In [26]:
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import accuracy_score

In [27]:
cv_results_KNNR = cross_validate(KNNR, x,y, cv=10, return_train_score=True, n_jobs=-1, scoring=('accuracy', 'precision', 'recall'))
cv_results_KNNR

{'fit_time': array([0.22686291, 0.25584292, 0.24285269, 0.26483774, 0.21786737,
        0.25584316, 0.23585439, 0.22985983, 0.22986102, 0.25784349]),
 'score_time': array([19.68526673, 22.05430627, 21.86442089, 22.33313417, 21.47766161,
        21.90839624, 19.41509104, 19.75054598, 19.66659927, 21.63256431]),
 'test_accuracy': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_precision': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'train_precision': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'test_recall': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'train_recall': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}

In [28]:
# from sklearn.pipeline import Pipeline
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
# from xgboost import XGBClassifier
# from sklearn.svm import SVC

# bagged_svc = BaggingClassifier(base_estimator = SVC(kernel = 'rbf', C = 10, gamma = 100), n_estimators = 100)

# _pipeline = Pipeline(steps = [
#                               ('encoder', FunctionTransformer(encode)), 
#                               ('pca', PCA()), 
# #                               ('scaler', StandardScaler()), 
# #                               ('model', bagged_svc),])
# #                               ('model', SVC(kernel = 'rbf', C = 10, gamma = 100)),])
#                               ('model', RandomForestRegressor(n_estimators = 1000, n_jobs = 9, max_depth = 20, min_samples_leaf = 1, min_samples_split = 2, criterion = 'entropy', random_state = 42))])
# #                               ('model', LogisticRegression(C = 1.0, solver = 'saga', penalty = 'l2', max_iter = 1000, random_state = 42))])

NameError: name 'FunctionTransformer' is not defined

In [ ]:
# from imblearn.pipeline import make_pipeline
# from imblearn.over_sampling import SMOTENC


# smotenc = SMOTENC(categorical_features = [0], k_neighbors = 5, random_state = 42)
# pipeline = make_pipeline(smotenc, _pipeline)
# pipeline